# Note:
The geographic locations of participating households, used to conduct spatial analyses, cannot be publicly available due to data privacy and ethical considerations.

In [1]:
import os
from pathlib import Path
from os import path

import geopandas as gpd
import numpy as np
import pandas as pd
import pysal
import scipy.stats as stats
import statsmodels.api as sm

from pysal.lib import weights
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. Input data

In [2]:
cwd = os.getcwd()
root_dir = Path(cwd).parent

In [3]:
## Survey data

# Individual data
data_IND_tot = pd.read_csv(path.join(root_dir,
                                    "data/raw/individual_data.csv"))

# Household data
data_HH_tot = pd.read_csv(path.join(root_dir,
                                    "data/raw/household_data.csv"))

## Geodata (confidential, not publicly accessible)

#Azito
azito = gpd.read_file(path.join(root_dir,
                                "data_confidential/preprocessed/data_AZ_HH_and_DiarrCases_blr.gpkg"))
azito = azito.to_crs(32630)

#Williamsville
willy = gpd.read_file(path.join(root_dir,
                                "data_confidential/preprocessed/data_WI_HH_and_DiarrCases_blr.gpkg"))
willy = willy.to_crs(32630)

# 2. Preprocess data

## 2.1. Join household attributes to individuals

In [4]:
## Join tables

# Dummy dataframes
dst_data = data_IND_tot.copy().drop(['Unnamed: 0'],axis=1)
src_data = data_HH_tot.copy().drop(['Unnamed: 0','SurveyArea','Site','uuid'],axis=1)
src_data = src_data.rename(columns={"KEY": "PARENT_KEY"})

# Join
dst_data = dst_data.merge(src_data, how='left', on='PARENT_KEY')

# Check results
dst_data

,KEY,PARENT_KEY,Site,SurveyArea,Relation_to_HH,Age,Sex,School_past,Diarrhoea,HH_Number,...,HHITEMS/Smartphone,HHITEMS/Television,HHITEMS/Noitems,HHITEMS/NA,HHEXTWALLS,HHROOF,HHFLOOR,HHROOMS,HHCOOKING,StreetFood
0,uuid:464423ec-b7a0-4924-b3e4-ce85bb6a5286/Rep_...,uuid:464423ec-b7a0-4924-b3e4-ce85bb6a5286,Azito,AZ2,Head,47.0,M,Secondary_2,N,252,...,0,1,0,0,Covered_adobe,CementFiber,Cement,1,Outdoors,0_1
1,uuid:464423ec-b7a0-4924-b3e4-ce85bb6a5286/Rep_...,uuid:464423ec-b7a0-4924-b3e4-ce85bb6a5286,Azito,AZ2,S_D,20.0,M,Secondary_2,N,252,...,0,1,0,0,Covered_adobe,CementFiber,Cement,1,Outdoors,0_1
2,uuid:b08a7cd9-67cf-45a0-9552-26956400cf7a/Rep_...,uuid:b08a7cd9-67cf-45a0-9552-26956400cf7a,Azito,AZ4,Head,58.0,M,High_Ed,N,339,...,1,1,0,0,Covered_adobe,CeramicTiles,Cement,2,House,2_4
3,uuid:b08a7cd9-67cf-45a0-9552-26956400cf7a/Rep_...,uuid:b08a7cd9-67cf-45a0-9552-26956400cf7a,Azito,AZ4,W_H,45.0,F,Secondary_2,Y,339,...,1,1,0,0,Covered_adobe,CeramicTiles,Cement,2,House,2_4
4,uuid:b08a7cd9-67cf-45a0-9552-26956400cf7a/Rep_...,uuid:b08a7cd9-67cf-45a0-9552-26956400cf7a,Azito,AZ4,GndC,3.0,F,NaN,N,339,...,1,1,0,0,Covered_adobe,CeramicTiles,Cement,2,House,2_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,uuid:57ef0f03-5ebd-44a5-b058-e9f26e0733d9/Rep_...,uuid:57ef0f03-5ebd-44a5-b058-e9f26e0733d9,Williamsville,W3,W_H,35.0,F,Primary,N,187,...,0,1,0,0,Covered_adobe,Metal,Ceramic,2,Outdoors,0_1
2494,uuid:57ef0f03-5ebd-44a5-b058-e9f26e0733d9/Rep_...,uuid:57ef0f03-5ebd-44a5-b058-e9f26e0733d9,Williamsville,W3,S_D,16.0,M,Secondary_2,N,187,...,0,1,0,0,Covered_adobe,Metal,Ceramic,2,Outdoors,0_1
2495,uuid:57ef0f03-5ebd-44a5-b058-e9f26e0733d9/Rep_...,uuid:57ef0f03-5ebd-44a5-b058-e9f26e0733d9,Williamsville,W3,S_D,12.0,M,Primary,N,187,...,0,1,0,0,Covered_adobe,Metal,Ceramic,2,Outdoors,0_1
2496,uuid:57ef0f03-5ebd-44a5-b058-e9f26e0733d9/Rep_...,uuid:57ef0f03-5ebd-44a5-b058-e9f26e0733d9,Williamsville,W3,S_D,6.0,M,Primary,N,187,...,0,1,0,0,Covered_adobe,Metal,Ceramic,2,Outdoors,0_1


## 2.2. Set exposure variables and subsets (age groups)

In [5]:
## Set exposure variables necessary for analysis: water facilities' characteristics + potential confounders

# Dummy dataframe
df = dst_data.copy()


## Outcome of interest
diarr_mask = (df["Diarrhoea"]=='Y')
df['Case'] = np.nan
df['Case'][(~df["Diarrhoea"].isna())] = 0
df['Case'][diarr_mask] = 1


## Water service

# Recode access to "basic water" (following JMP definition) VS any other service type (including unimproved)
wt_basic_mask = (((df["DrinkingWaterGroup"]=='Private')&
                  (df["DrinkingWater_Private"].isin(['Piped_dwel','Piped_yard'])))|
                 ((df["DrinkingWaterGroup"]=='Private')&
                  (df["DrinkingWater_Private"].isin(['Piped_neigh']))&
                  (df["DrinkingWaterDist"]<=30))|
                 ((df["DrinkingWaterGroup"]=='Public')&
                  (df["DrinkingWater_Public"].isin(['Vendor','Ground_tube']))&
                  (df["DrinkingWaterDist"]<=30)))
df["BasicWt"] = np.nan
df["BasicWt"][~df["DrinkingWaterGroup"].isna()] = 0
df["BasicWt"][wt_basic_mask] = 1

# Recode access to SODECI subsidized ("social") connection (water piped to premises) VS any other service type (including unimproved)
wt_soc_mask = ((df["DrinkingWater_Private"].isin(['Piped_dwel','Piped_yard']))&
               (df["DrinkingWaterPaySocial"]=="Y"))
df["SubsidWtCn"] = np.nan
df["SubsidWtCn"][(~df["DrinkingWaterGroup"].isna())] = 0
df["SubsidWtCn"][wt_soc_mask] = 1

# Recode water availability in the past month (self-reported)
wt_avail_mask = (df["DrinkingWaterAvailability"]=='N') # no lack of water in the previous month
df["AvailblWtM"] = np.nan
df["AvailblWtM"][~df["DrinkingWaterAvailability"].isna()] = 0
df["AvailblWtM"][wt_avail_mask] = 1


## Sanitation service

# Recode sanitation variable: "basic" (according to WHO-UNICEF's JMP) VS any other facitilty type (including unimproved and O.D.)
basic_sn_mask = ((((df.ToiletFacilityTYPE=='DryOrCompost')&
                   (df.ToiletFacilityTYPE_Dry.isin(['Dry_ImprSlab',
                                                    'Dry_VIP'])))|
                  ((df.ToiletFacilityTYPE=='Flush')&
                   (df.ToiletFacilityTYPE_Flush.isin(['Flush_piped',
                                                      'Flush_septic_tank',
                                                      'Flush_coveredPit']))))&
                 (df.ToiletROOF=='Y')&
                 (df.ToiletFacilitySHARE=='N'))
df["BasicSan"] = np.nan
df["BasicSan"][(~df.ToiletFacility.isna())] = 0
df["BasicSan"][basic_sn_mask] = 1


## Hygiene service

# Recode "basic" hygiene (according to WHO-UNICEF's JMP)
basic_hg_mask = ((df["ObsHandWashWATER"]=='Water_OK')& # water available at the moment of the survey
                 (df["ObsHandWashSOAP"]=='SoapOrDeterg')& # soap (or equivalent) available at the moment of the survey
                 (df["ObsHandWashPLACE"].isin(['Obs_Fixed','Obs_Mobile']))) # presence of amenity to wash hands
df["BasicHyg"] = np.nan
df["BasicHyg"][(~df["ObsHandWashWATER"].isna())&(~df["ObsHandWashSOAP"].isna())&(~df["ObsHandWashPLACE"].isna())] = 0
df["BasicHyg"][basic_hg_mask] = 1


## Housing conditions

# Recode cooking area (whether cooking is done inside dwelling or elsewhere)
cook_mask = (df["HHCOOKING"]=='House')# cooking done inside dwelling
df["CookInsd"] = np.nan
df["CookInsd"][~df["HHCOOKING"].isna()] = 0
df["CookInsd"][cook_mask] = 1

# Recode precarious housing (based on dwelling's materials)
prec_h_mask = ((df["HHEXTWALLS"].isin(['Stone_mud','ReusedWoodOrBamboo','Metal']))|
               (df["HHFLOOR"].isin(['Other']))|
               (df["HHROOF"].isin(['Rustic_metal','Rustic_toles_fibr','Rustic_planks','Rustic_plastic'])))
df["PrecHHMat"] = np.nan
df["PrecHHMat"][(~df["HHEXTWALLS"].isna())&(~df["HHFLOOR"].isna())&(~df["HHROOF"].isna())] = 0
df["PrecHHMat"][prec_h_mask] = 1

# Recode overcrowding (whether number of habitants per room > 3, following 'slum' definition in SDG 11)
hab_cnt_df = df[['KEY','PARENT_KEY']].groupby('PARENT_KEY').count().reset_index() # count habitants per household
hab_cnt_df = hab_cnt_df.rename(columns={"KEY": "Cnt_Hab"})
df = df.merge(hab_cnt_df,how='left',on='PARENT_KEY') # join variable with count of houshehold members
df['HHROOMS'][df['HHROOMS'].isin([111,999])] = np.nan # exclude invalid answers & outliers regarding rooms
df['HabPerRoom'] = df['Cnt_Hab']/df['HHROOMS']
hiden_mask = (df["HabPerRoom"]>3) # high density if number of household members per room > 3
df["HiDensity"] = np.nan
df["HiDensity"][~df["HabPerRoom"].isna()] = 0
df["HiDensity"][hiden_mask] = 1
hiden_mask2 = (df["HabPerRoom"]>np.nanquantile(df["HabPerRoom"],0.75)) # high density if number of household members per room > last quartile (i.e., 4)
df["HiDensity2"] = np.nan
df["HiDensity2"][~df["HabPerRoom"].isna()] = 0
df["HiDensity2"][hiden_mask2] = 1


## Habitat (spatial analysis addressing characteristics of neighbouring observations)

# Obtain neighbouring habitat's characteristics
gdf_list = [azito,willy]
dist_bw = 100 # parameter for distance-based weight matrix: bandwith = 100 m
#n_neighbours = 8 # parameter for KNN weight matrix: n = 8 closest neighbours
for gdf in gdf_list:
    # Recode sanitation variable: "basic" (according to WHO-UNICEF's JMP) VS any other facitilty type (including unimproved and O.D.)
    basic_sn_mask = ((((gdf.ToiletFacilityTYPE=='DryOrCompost')&
                       (gdf.ToiletFacilityTYPE_Dry.isin(['Dry_ImprSlab',
                                                         'Dry_VIP'])))|
                      ((gdf.ToiletFacilityTYPE=='Flush')&
                       (gdf.ToiletFacilityTYPE_Flush.isin(['Flush_piped',
                                                           'Flush_septic_tank',
                                                           'Flush_coveredPit']))))&
                     (gdf.ToiletROOF=='Y')&
                     (gdf.ToiletFacilitySHARE=='N'))
    gdf["BasicSan"] = np.nan
    gdf["BasicSan"][(~gdf.ToiletFacility.isna())] = 0
    gdf["BasicSan"][basic_sn_mask] = 1
    # Set up distance weights matrix, based on point geodataframe
    #wq = weights.KNN.from_dataframe(gdf, k=n_neighbours)
    wq = weights.DistanceBand.from_dataframe(gdf, threshold=dist_bw)
    wq.transform = 'r' # standardize matrix (divide by number of neighboring observations)
    # Relevant habitat exposure variables
    exposure_lst = ['BasicSan', # access to basic sanitation
                   ]
    # Calculate proportion of neighbours (within distance bandwith defined above) having a positive outcome, for each exposure variable
    lagged_var_list = []
    for var in exposure_lst:
        lag_var_name = var+'_lag'
        lagged_var_list = lagged_var_list+[lag_var_name]
        gdf[lag_var_name] = np.nan
        gdf[lag_var_name] = weights.spatial_lag.lag_spatial(wq, gdf[var]) # row-standardised weights style: mean of neighbors

# Unify geodataframes
var_list = ['PARENT_KEY']+lagged_var_list
gdf_lagged = azito[var_list].append(willy[var_list],ignore_index=True)

# Join lagged data to individual data
df = df.merge(gdf_lagged,on='PARENT_KEY',how='left')

# Variable indicating neighbours' coverage of basic sanitation
threshold = 0.75 # collective sanitation coverage threshold based on FAECI index (doi.org/10.1016/j.ijheh.2018.11.005)
lc_san_mask = (df["BasicSan_lag"]>=threshold) # local basic sanitation: neighbours' coverage of basic sanitation >= 75%
df["BasicSanLC"] = np.nan
df["BasicSanLC"][(~df["BasicSan_lag"].isna())] = 0
df["BasicSanLC"][lc_san_mask] = 1


## External contamination pathways

# Recode frequency of consumption of street food
st_food_mask = (df["StreetFood"].isin(['2_4','5_Or_More'])) # twice or more per week
df["StFood"] = np.nan
df["StFood"][(~df["StreetFood"].isna())] = 0
df["StFood"][st_food_mask] = 1


## Potential confounders

# Wealth based on households assets
df["ItemsScore"]=(df["HHITEMS/Computer"]+df["HHITEMS/Electricity"]+
                      df["HHITEMS/Internet"]+df["HHITEMS/Oven"]+
                      df["HHITEMS/Radio"]+df["HHITEMS/Refrigerator"]+
                      df["HHITEMS/Smartphone"]+df["HHITEMS/Television"])
df["ItemsScore"][(df["HHITEMS/Noitems"]==1)] = 0
df["ItemsScore"][(df["HHITEMS/NA"]==1)] = np.nan
wt_hdhyg_mask = (df["ItemsScore"]>df["ItemsScore"].mean())# Number of items above the mean (of the surveyed households)
df["WealthyHH"] = np.nan
df["WealthyHH"][~df["ItemsScore"].isna()] = 0
df["WealthyHH"][wt_hdhyg_mask] = 1

# Education level of heads of households
src = data_IND_tot[data_IND_tot.Relation_to_HH=='Head'][['PARENT_KEY',
                                                         'School_past']]# subset ed. level of HH
# Recode education level of heads of households
src['SecEduHH'] = np.nan
src['SecEduHH'][src['School_past'].isin(['No_Edu',
                                         'Early_CdE',
                                         'Primary',
                                         'Coranic'])] = 0 # up to primary education
src['SecEduHH'][src['School_past'].isin(['Secondary',
                                         'Secondary_1',
                                         'Secondary_2',
                                         'High_Ed'])] = 1 # secondary or higher education
src = src[~src.SecEduHH.isna()]
# In case a same household has 2 heads, merge lines
src = src[['SecEduHH','PARENT_KEY']].groupby(by="PARENT_KEY").max().reset_index()
# Attribute education level of heads of households
df = df.merge(src,on="PARENT_KEY",how='left')

In [6]:
## Subsets

# General population
dst_data = df.copy()
print("N for general pop.:",dst_data.shape[0])

# Under 5, Abidjan
mask = (dst_data["Age"]<5)
dst_data_U5 = dst_data[mask]
print("N for under 5:",dst_data_U5.shape[0])

N for general pop.: 2498
N for under 5: 283


# 3. Descriptive statistics

In [7]:
## Number of individuals

# Both sites
tot_ind = df.shape[0]
tot_hh = len(df.PARENT_KEY.unique())

# Azito
tot_ind_az = df[df['Site']=='Azito'].shape[0]
tot_hh_az = len(df[df['Site']=='Azito'].PARENT_KEY.unique())

# Williamsville
tot_ind_wi = df[df['Site']=='Williamsville'].shape[0]
tot_hh_wi = len(df[df['Site']=='Williamsville'].PARENT_KEY.unique())

# Print stats
print('Total individuals:',tot_ind)
print('Total households:',tot_hh)
print('----------------------------------------')
print('Total individuals in Azito:',tot_ind_az)
print('Total households in Azito:',tot_hh_az)
print('----------------------------------------')
print('Total individuals in Williamsville:',tot_ind_wi)
print('Total households in Williamsville:',tot_hh_wi)

Total individuals: 2498
Total households: 567
----------------------------------------
Total individuals in Azito: 1191
Total households in Azito: 266
----------------------------------------
Total individuals in Williamsville: 1307
Total households in Williamsville: 301


In [8]:
## Number of individuals, by gender

# Both sites
tot_ind_m = df[df['Sex']=='M'].shape[0]
tot_ind_f = df[df['Sex']=='F'].shape[0]
tot_ind_x= df[df['Sex'].isna()].shape[0]

# Azito
tot_ind_az_m = df[(df['Sex']=='M')&(df['Site']=='Azito')].shape[0]
tot_ind_az_f = df[(df['Sex']=='F')&(df['Site']=='Azito')].shape[0]
tot_ind_az_x = df[(df['Sex'].isna())&(df['Site']=='Azito')].shape[0]

# Williamsville
tot_ind_wi_m = df[(df['Sex']=='M')&(df['Site']=='Williamsville')].shape[0]
tot_ind_wi_f = df[(df['Sex']=='F')&(df['Site']=='Williamsville')].shape[0]
tot_ind_wi_x = df[(df['Sex'].isna())&(df['Site']=='Williamsville')].shape[0]

# Print stats
print('Total male individuals:',tot_ind_m,
      '(',round((tot_ind_m/tot_ind)*100,2),'% )')
print('Total female individuals:',tot_ind_f,
      '(',round((tot_ind_f/tot_ind)*100,2),'% )')
print('Total individuals without data on sex:',tot_ind_x,
      '(',round((tot_ind_x/tot_ind)*100,2),'% )')
print('----------------------------------------')
print('Total male individuals in Azito:',tot_ind_az_m,
      '(',round((tot_ind_az_m/tot_ind_az)*100,2),'% )')
print('Total female individuals in Azito:',tot_ind_az_f,
      '(',round((tot_ind_az_f/tot_ind_az)*100,2),'% )')
print('----------------------------------------')
print('Total male individuals in Williamsville:',tot_ind_wi_m,
      '(',round((tot_ind_wi_m/tot_ind_wi)*100,2),'% )')
print('Total female individuals in Williamsville:',tot_ind_wi_f,
      '(',round((tot_ind_wi_f/tot_ind_wi)*100,2),'% )')

Total male individuals: 1170 ( 46.84 % )
Total female individuals: 1328 ( 53.16 % )
Total individuals without data on sex: 0 ( 0.0 % )
----------------------------------------
Total male individuals in Azito: 543 ( 45.59 % )
Total female individuals in Azito: 648 ( 54.41 % )
----------------------------------------
Total male individuals in Williamsville: 627 ( 47.97 % )
Total female individuals in Williamsville: 680 ( 52.03 % )


In [9]:
## Number of individuals, by age

# Both sites
tot_ind_ag_u5 = df[df['Age']<5].shape[0]
tot_ind_ag_ad = df[(df['Age']>18)&(df['Age']<=90)].shape[0]
tot_ind_ag_x = df[(df['Age'].isna())|(df['Age']>90)].shape[0]

# Azito
tot_ind_az_ag_u5 = df[(df['Age']<5)&(df['Site']=='Azito')].shape[0]
tot_ind_az_ag_ad = df[(df['Age']>18)&(df['Age']<=90)&(df['Site']=='Azito')].shape[0]
tot_ind_az_ag_x = df[((df['Age'].isna())|(df['Age']>90))&(df['Site']=='Azito')].shape[0]

# Williamsville
tot_ind_wi_ag_u5 = df[(df['Age']<5)&(df['Site']=='Williamsville')].shape[0]
tot_ind_wi_ag_ad = df[(df['Age']>18)&(df['Age']<=90)&(df['Site']=='Williamsville')].shape[0]
tot_ind_wi_ag_x = df[((df['Age'].isna())|(df['Age']>90))&(df['Site']=='Williamsville')].shape[0]

# Print stats
print('Total individuals under five:',tot_ind_ag_u5,
      '(',round((tot_ind_ag_u5/tot_ind)*100,2),'% )')
print('Total adult individuals:',tot_ind_ag_ad,
      '(',round((tot_ind_ag_ad/tot_ind)*100,2),'% )')
print('Total individuals without data on age:',tot_ind_ag_x,
      '(',round((tot_ind_ag_x/tot_ind)*100,2),'% )')
print('----------------------------------------')
print('Total individuals under five in Azito:',tot_ind_az_ag_u5,
      '(',round((tot_ind_az_ag_u5/tot_ind_az)*100,2),'% )')
print('Total adult individuals in Azito:',tot_ind_az_ag_ad,
      '(',round((tot_ind_az_ag_u5/tot_ind_az)*100,2),'% )')
print('Total individuals without data on age in Azito:',tot_ind_az_ag_x,
      '(',round((tot_ind_az_ag_x/tot_ind_az)*100,2),'% )')
print('----------------------------------------')
print('Total individuals under five in Williamsville:',tot_ind_wi_ag_u5,
      '(',round((tot_ind_wi_ag_u5/tot_ind_wi)*100,2),'% )')
print('Total adult individuals in Williamsville:',tot_ind_wi_ag_ad,
      '(',round((tot_ind_wi_ag_u5/tot_ind_wi)*100,2),'% )')
print('Total individuals without data on age in Williamsville:',tot_ind_wi_ag_x,
      '(',round((tot_ind_wi_ag_x/tot_ind_wi)*100,2),'% )')

Total individuals under five: 283 ( 11.33 % )
Total adult individuals: 1245 ( 49.84 % )
Total individuals without data on age: 238 ( 9.53 % )
----------------------------------------
Total individuals under five in Azito: 124 ( 10.41 % )
Total adult individuals in Azito: 599 ( 10.41 % )
Total individuals without data on age in Azito: 134 ( 11.25 % )
----------------------------------------
Total individuals under five in Williamsville: 159 ( 12.17 % )
Total adult individuals in Williamsville: 646 ( 12.17 % )
Total individuals without data on age in Williamsville: 104 ( 7.96 % )


# 4. Odds ratios from multiple logistic regressions

## 4.1. Exploratory analysis to identify candidate explanatory variables (to be included in logistic model)

### 4.1.1. Bivariate odds ratio analysis to identify candidate explanatory variables for diarrhoea

In [10]:
## Set list of exposure variables

# List
exposure_lst = [# WASH:
                'SubsidWtCn', # whether household has subsidized connection to water network
                'AvailblWtM', # whether water was available throughout the month preceding the survey (self reported)
                'BasicSanLC', # whether neighbours' coverage of basic sanitation is >= 75%
                # Housing conditions:
                'PrecHHMat', # whether household is built with inadequate materials (observed)
                'HiDensity', # whether household is overcrowded (normative definition, i.e. >3 per room)
                'HiDensity2', # whether household is overcrowded (relative  definition, i.e. > series' last quartile)
                'CookInsd', # whether household has an indoors cooking space (inside dwelling)
                # External factors
                'StFood', # frequent consumption of street foods (twice or more per week)
                # Potential confounders:
                'SecEduHH', # whether head of household has secondary education
                'WealthyHH', # whether household has more assets than the average
               ]

# Subsets list 1: general population
subsets_gen = [dst_data]
subsets_gen_str = ['general population']

# Subsets list 2: households with children under five years old
subsets_cU5 = [dst_data_U5]
subsets_cU5_str = ['children under five']

In [11]:
## Risk of diarrhoea, general population

# Calculate odds ratios
outcome_var = "Diarrhoea"
outcome_pos = "Y"
outcome_neg = "N"
df_oddsr_gen = pd.DataFrame()
for idx, subset in enumerate(subsets_gen):
    print("------------------ ",subsets_gen_str[idx]," ------------------")
    for exposure in exposure_lst:
        # define groups
        print("Variable:",exposure)
        exposure_grp = subset[subset[exposure]==1]
        no_exposure_grp = subset[subset[exposure]==0]
        exposure_grp = exposure_grp[~exposure_grp[outcome_var].isna()]
        no_exposure_grp = no_exposure_grp[~no_exposure_grp[outcome_var].isna()]
        # set table for Fisher tests
        table = np.array([[exposure_grp[exposure_grp[outcome_var]==outcome_pos].shape[0], exposure_grp[exposure_grp[outcome_var]==outcome_neg].shape[0]],
                          [no_exposure_grp[no_exposure_grp[outcome_var]==outcome_pos].shape[0], no_exposure_grp[no_exposure_grp[outcome_var]==outcome_neg].shape[0]]])
        # calculate proportion of households with at least 1 case
        exposure_grp_prop = exposure_grp[exposure_grp[outcome_var]==outcome_pos].shape[0]/exposure_grp.shape[0]
        no_exposure_grp_prop = no_exposure_grp[no_exposure_grp[outcome_var]==outcome_pos].shape[0]/no_exposure_grp.shape[0]
        # calculate 95% CI - exposure group
        P_exp = exposure_grp_prop
        N_exp = exposure_grp.shape[0]
        CI_exp = 1.96*(np.sqrt((P_exp*(1-P_exp))/N_exp))
        # calculate 95% CI - no exposure group
        P_ne = no_exposure_grp_prop
        N_ne = no_exposure_grp.shape[0]
        CI_ne = 1.96*(np.sqrt((P_ne*(1-P_ne))/N_ne))
        # run Fisher tests for OR = 1
        oddsratio_eq1, pvalue_eq1 = stats.fisher_exact(table)
        # run Fisher tests for OR > 1
        oddsratio_greater1, pvalue_greater1 = stats.fisher_exact(table,alternative="greater")
        # run Fisher tests for OR < 1
        oddsratio_less1, pvalue_less1 = stats.fisher_exact(table,alternative="less")
        # add results to dataframe
        df_oddsr_gen = df_oddsr_gen.append([[exposure,
                                             subsets_gen_str[idx],
                                             exposure_grp_prop*100,
                                             '±'+str(round(CI_exp*100,2)),
                                             no_exposure_grp_prop*100,
                                             '±'+str(round(CI_ne*100,2)),
                                             oddsratio_eq1,
                                             pvalue_eq1,
                                             pvalue_greater1,
                                             pvalue_less1,
                                             table
                                            ]])

# Reset columns' names & index
df_oddsr_gen.columns = ['exposure variable','stratum',
                        '% outcomes, exposed','95% CI, exposed',
                        '% outcomes, non-exposed','95% CI, non-exposed',
                        'OR','p-value for OR=1',
                        'p_OR_hi_1','p_OR_lo_1',
                        'table'
                       ]
df_oddsr_gen = df_oddsr_gen.reset_index()

------------------  general population  ------------------
Variable: SubsidWtCn
Variable: AvailblWtM
Variable: BasicSanLC
Variable: PrecHHMat
Variable: HiDensity
Variable: HiDensity2
Variable: CookInsd
Variable: StFood
Variable: SecEduHH
Variable: WealthyHH


In [12]:
## Risk of diarrhoea, under fives subsets

# Calculate odds ratios
outcome_var = "Diarrhoea"
outcome_pos = "Y"
outcome_neg = "N"
df_oddsr_cU5 = pd.DataFrame()
for idx, subset in enumerate(subsets_cU5):
    print("------------------ ",subsets_cU5_str[idx]," ------------------")
    for exposure in exposure_lst:
        # define groups
        print("Variable:",exposure)
        exposure_grp = subset[subset[exposure]==1]
        no_exposure_grp = subset[subset[exposure]==0]
        exposure_grp = exposure_grp[~exposure_grp[outcome_var].isna()]
        no_exposure_grp = no_exposure_grp[~no_exposure_grp[outcome_var].isna()]
        # set table for Fisher tests
        table = np.array([[exposure_grp[exposure_grp[outcome_var]==outcome_pos].shape[0], exposure_grp[exposure_grp[outcome_var]==outcome_neg].shape[0]],
                          [no_exposure_grp[no_exposure_grp[outcome_var]==outcome_pos].shape[0], no_exposure_grp[no_exposure_grp[outcome_var]==outcome_neg].shape[0]]])
        # calculate proportion of households with at least 1 case
        exposure_grp_prop = exposure_grp[exposure_grp[outcome_var]==outcome_pos].shape[0]/exposure_grp.shape[0]
        no_exposure_grp_prop = no_exposure_grp[no_exposure_grp[outcome_var]==outcome_pos].shape[0]/no_exposure_grp.shape[0]
        # calculate 95% CI - exposure group
        P_exp = exposure_grp_prop
        N_exp = exposure_grp.shape[0]
        CI_exp = 1.96*(np.sqrt((P_exp*(1-P_exp))/N_exp))
        # calculate 95% CI - no exposure group
        P_ne = no_exposure_grp_prop
        N_ne = no_exposure_grp.shape[0]
        CI_ne = 1.96*(np.sqrt((P_ne*(1-P_ne))/N_ne))
        # run Fisher tests for OR = 1
        oddsratio_eq1, pvalue_eq1 = stats.fisher_exact(table)
        # run Fisher tests for OR > 1
        oddsratio_greater1, pvalue_greater1 = stats.fisher_exact(table,alternative="greater")
        # run Fisher tests for OR < 1
        oddsratio_less1, pvalue_less1 = stats.fisher_exact(table,alternative="less")
        # add results to dataframe
        df_oddsr_cU5 = df_oddsr_cU5.append([[exposure,
                                             subsets_cU5_str[idx],
                                             exposure_grp_prop*100,
                                             '±'+str(round(CI_exp*100,2)),
                                             no_exposure_grp_prop*100,
                                             '±'+str(round(CI_ne*100,2)),
                                             oddsratio_eq1,
                                             pvalue_eq1,
                                             pvalue_greater1,
                                             pvalue_less1,
                                             table
                                            ]])

# Reset columns' names
df_oddsr_cU5.columns = ['exposure variable','stratum',
                        '% outcomes, exposed','95% CI, exposed',
                        '% outcomes, non-exposed','95% CI, non-exposed',
                        'OR','p-value for OR=1',
                        'p_OR_hi_1','p_OR_lo_1',
                        'table'
                       ]
df_oddsr_cU5 = df_oddsr_cU5.reset_index()

------------------  children under five  ------------------
Variable: SubsidWtCn
Variable: AvailblWtM
Variable: BasicSanLC
Variable: PrecHHMat
Variable: HiDensity
Variable: HiDensity2
Variable: CookInsd
Variable: StFood
Variable: SecEduHH
Variable: WealthyHH


In [13]:
## List of significant covariates

# Significance threshold
st = 0.1

# General population
list_gen = df_oddsr_gen['exposure variable'][df_oddsr_gen['p-value for OR=1']<st].to_list()

# Under fives subset
list_cu5 = df_oddsr_cU5['exposure variable'][df_oddsr_cU5['p-value for OR=1']<st].to_list()

# Merge lists
merged_lst = list(set(list_gen+list_cu5))

# Dataframe containing only selected covariates, by age group
df_raw_or = df_oddsr_gen[df_oddsr_gen['exposure variable'].isin(merged_lst)].append(df_oddsr_cU5[df_oddsr_cU5['exposure variable'].isin(merged_lst)]).drop(['p_OR_hi_1', 'p_OR_lo_1'],axis=1).reset_index().drop(['level_0', 'index'],axis=1)

# Check results
print("Selected covariates (n=",len(merged_lst),"):",merged_lst)
df_raw_or

Selected covariates (n= 5 ): ['AvailblWtM', 'SecEduHH', 'PrecHHMat', 'WealthyHH', 'CookInsd']


,exposure variable,stratum,"% outcomes, exposed","95% CI, exposed","% outcomes, non-exposed","95% CI, non-exposed",OR,p-value for OR=1,table
0,AvailblWtM,general population,12.726138,±1.63,17.887155,±2.6,0.669395,8.263162e-04,"[[204, 1399], [149, 684]]"
1,PrecHHMat,general population,23.123732,±3.72,12.287918,±1.46,2.147073,6.963739e-09,"[[114, 379], [239, 1706]]"
2,CookInsd,general population,11.517241,±1.64,18.825911,±2.44,0.561243,7.608208e-07,"[[167, 1283], [186, 802]]"
3,SecEduHH,general population,16.568627,±2.28,13.405797,±2.01,1.282783,4.428440e-02,"[[169, 851], [148, 956]]"
4,WealthyHH,general population,16.495601,±1.97,11.918063,±1.94,1.459957,1.422630e-03,"[[225, 1139], [128, 946]]"
5,AvailblWtM,children under five,22.702703,±6.04,28.125000,±8.99,0.750583,3.806429e-01,"[[42, 143], [27, 69]]"
6,PrecHHMat,children under five,38.709677,±12.12,20.547945,±5.35,2.442105,4.562482e-03,"[[24, 38], [45, 174]]"
7,CookInsd,children under five,20.270270,±6.48,29.323308,±7.74,0.612777,9.569732e-02,"[[30, 118], [39, 94]]"
8,SecEduHH,children under five,31.481481,±8.76,21.739130,±6.88,1.654054,1.069624e-01,"[[34, 74], [30, 108]]"
9,WealthyHH,children under five,26.206897,±7.16,22.794118,±7.05,1.202894,5.794909e-01,"[[38, 107], [31, 105]]"


### 4.1.2. Confirm selection of independent variables: test for multicollinearity with Variance Inflation Factor
Note: as a thumbrule, variables with a VIF > 5 shall be discarded 

In [14]:
# Set VIF threshold
vif_t = 5

# Set list of selected exposure + control variables
exposure_list = merged_lst.copy()
control_list = ['BasicWt', # access to basic water service
                'BasicSan', # access to basic sanitation
                'BasicHyg', # presence of hand-washing materials AND fixed structure to wash hands (observed)
                ]

# Dummy dataframes: keep only observations with valid answers for all selected variables
dataset = dst_data[control_list+exposure_list].copy().dropna()

# Create table for Variance Inflation Factor (VIF)
vif_scores = pd.DataFrame() 
vif_scores["Attribute"] = control_list+exposure_list

# Calculate VIF for each feature
vif_var = "VIF Scores"
vif_scores[vif_var] = [variance_inflation_factor(dataset[control_list+exposure_list].values, i) for i in range(len(dataset[control_list+exposure_list].columns))] 

# Assess variables' VIF
if vif_scores[vif_var].max() < vif_t:
    print("All good: no significant multicollinearity")
else:
    drop_v_list = vif_scores["Attribute"][vif_scores[vif_var]>vif_t].to_list()
    print(">> WARNING: multicollinearity detected for:",
          drop_v_list)
    # Update list of selected variables
    merged_lst = list(set(merged_lst)-set(drop_v_list))
    print("Verified covariates (n=",len(merged_lst),"):",merged_lst)

# View results
display(vif_scores)

>> WARNING: multicollinearity detected for: ['BasicWt']
Verified covariates (n= 5 ): ['AvailblWtM', 'SecEduHH', 'PrecHHMat', 'WealthyHH', 'CookInsd']


,Attribute,VIF Scores
0,BasicWt,8.175330
1,BasicSan,2.627561
2,BasicHyg,2.760119
3,AvailblWtM,3.009245
4,SecEduHH,2.247989
5,PrecHHMat,1.562794
6,WealthyHH,2.579489
7,CookInsd,3.048534


## 4.2. Adjusted odds ratios (i) : multiple logistic regressions stratified by age group

In [15]:
# Set list of selected exposure variables + control variables (WASH)
control_list = [#'BasicWt', # access to basic water service EXClUDED due to high VIF and lack of variation (99% of all households had basic water)
                'BasicSan', # access to basic sanitation
                'BasicHyg', # presence of hand-washing materials AND fixed structure to wash hands (observed)
                ]
exposure_lst = sorted(control_list+merged_lst)

# Data
outcome = 'Case'
data_reg_genpp = dst_data[[outcome]+exposure_lst].copy().dropna()
data_reg_cu5 = dst_data_U5[[outcome]+exposure_lst].copy().dropna()
data_strata = ['general population ( n = '+str(data_reg_genpp.shape[0])+')',
               'under-fives subset ( n = '+str(data_reg_cu5.shape[0])+')']

# Create table for ORs (initially empty)
or_strage = pd.DataFrame()

# Discard exposure variable if all observations are exposed, or non-exposed (OR not plausible)
drop_list = []
print('==============================================================================')
print('                         CHECKING EXPOSURE VARIABLES:')
for data_reg in [data_reg_genpp,data_reg_cu5]:
    print('')
    for var in exposure_lst:
        if len(data_reg[var].value_counts().values)==1:
            drop_list = drop_list+[var]
            print('>> WARNING : dropped',var,'( only 0s or 1s in ',data_strata[idx],')')
if len(drop_list)==0:
    print('>> All good!')
elif len(drop_list)>0:
    exposure_lst = sorted(list(set(exposure_lst)-set(drop_list))) # update exposure variables list if needed
print('==============================================================================')

# Calculate stratified, adjusted ORs
for idx,data_reg in enumerate([data_reg_genpp,data_reg_cu5]):
    # Fit a logistic regression model with all the variables
    logit = sm.Logit(data_reg['Case'], sm.add_constant(data_reg[exposure_lst]))
    result = logit.fit()

    # Print the summary of the model
    print('==============================================================================')
    print('                       ',data_strata[idx])
    print(result.summary())

    # Calculate the odds ratios and confidence intervals
    params = result.params
    conf = result.conf_int()
    signif = result.pvalues
    modelpval = result.llr_pvalue
    odds_ratios = np.exp(params)
    conf_lower = np.exp(conf[0])
    conf_upper = np.exp(conf[1])

    # Print the odds ratios and confidence intervals
    or_strat = pd.DataFrame({'OR': odds_ratios,
                             'signif': signif,
                             'Lower CI': conf_lower,
                             'Upper CI': conf_upper}).reset_index()
    or_strat['Significance'] = ''
    if modelpval < 0.05:
        or_strat['Significance'][or_strat['signif']>=0.1] = 'Not significant'
        or_strat['Significance'][or_strat['signif']<0.1] = '*'
        or_strat['Significance'][or_strat['signif']<0.05] = '**'
        or_strat['Significance'][or_strat['signif']<0.01] = '***'
        or_strat['Significance'][or_strat['signif']<0.001] = '****'
    else:
        or_strat['Significance'] = 'Not significant (LLR p-value > 5%)'
    or_strat = or_strat.drop(['signif'],axis=1)
    multi_index = pd.MultiIndex.from_tuples([(data_strata[idx],'Exposure'),
                                             (data_strata[idx],'Adjusted OR'),
                                             (data_strata[idx],'Lower CI (95%)'),
                                             (data_strata[idx],'Upper CI (95%)'),
                                             (data_strata[idx],'Significance')])
    or_strat.columns = multi_index
    or_strat = or_strat.iloc[1:]
    or_strage = pd.concat([or_strage,or_strat],axis=1)

# See final OR table
print('==============================================================================')
print('                     Stratified, adjusted odds ratios :')
or_strage = or_strage.replace({'BasicWt':'Access to basic water service',
                               'BasicSan':'Access to basic sanitation service',
                               'BasicHyg':'Access to basic hygiene service',
                               'PrecHHMat':'Precarious dwelling materials',
                               'CookInsd':'Indoors cooking space',
                               'AvailblWtM':'Water availability (month preceding survey)',
                               'SecEduHH':'Head of household with secondary education',
                               'WealthyHH':'Relatively wealthy household',
                               'SubsidWtCn':'Subsidized water connection'
                              })
or_strage

                         CHECKING EXPOSURE VARIABLES:


>> All good!
Optimization terminated successfully.
         Current function value: 0.401354
         Iterations 6
                        general population ( n = 2043)
                           Logit Regression Results                           
Dep. Variable:                   Case   No. Observations:                 2043
Model:                          Logit   Df Residuals:                     2035
Method:                           MLE   Df Model:                            7
Date:                Fri, 07 Apr 2023   Pseudo R-squ.:                 0.05157
Time:                        12:31:36   Log-Likelihood:                -819.97
converged:                       True   LL-Null:                       -864.55
Covariance Type:            nonrobust   LLR p-value:                 1.836e-16
                 coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------

general population ( n = 2043)                             \
                                      Exposure Adjusted OR Lower CI (95%)   
1  Water availability (month preceding survey)    0.682411       0.525942   
2              Access to basic hygiene service    0.602684       0.457191   
3           Access to basic sanitation service    1.201792       0.893673   
4                        Indoors cooking space    0.552721       0.416086   
5                Precarious dwelling materials    1.790061       1.310652   
6   Head of household with secondary education    1.507353       1.148853   
7                 Relatively wealthy household    1.508920       1.146337   

                                   \
  Upper CI (95%)     Significance   
1       0.885428              ***   
2       0.794480             ****   
3       1.616145  Not significant   
4       0.734225             ****   
5       2.444829             ****   
6       1.977721              ***   
7       1.986186              ***   

                 under-fives subset ( n = 235)                             \
                                      Exposure Adjusted OR Lower CI (95%)   
1  Water availability (month preceding survey)    0.632775       0.325208   
2              Access to basic hygiene service    0.366363       0.182884   
3           Access to basic sanitation service    1.480264       0.731007   
4                        Indoors cooking space    0.700937       0.346167   
5                Precarious dwelling materials    2.055774       0.976300   
6   Head of household with secondary education    2.109066       1.089952   
7                 Relatively wealthy household    1.254144       0.627616   

                                   
  Upper CI (95%)     Significance  
1       1.231224  Not significant  
2       0.733919              ***  
3       2.997486  Not significant  
4       1.419292  Not significant  
5       4.328802                *  
6       4.081061               **  
7       2.506112  Not significant

In [16]:
## SAVE to CSV
or_strage.to_csv(path.join(root_dir,
                           "data/output/Notebook_1_aOR_strage.csv"))

# 5. Notes

### General
Globally, education and access to basic hygiene amenities at home were the only variables that were consistently and significantly associated with diarrhea accross age groups. The adjusted ORs (aORs) suggest that DEPRIVED LIVING ENVIRONMENTS (inadequate dwelling materials and lack of proper cooking spaces) EXACERBATE VULNERABILITY to diarrheal infections (e.g., by increasing exposure to environmental hazards), and must be ADDRESSED IN SINERGY with the extension of basic WASH services. In fact, even when controlling by onther relevant factors (sanitation and hygiene amenities) and potential confounders (socioeconomic variables), housing conditions were significantly associated with a higher risk of diarrhea in the general population. Moreover, the results in Notebook 1b show that almost all exposure variables associated with diarrhoea (identified here) are co-dependent on socioeconomic status. Hence, the risk of diarrhoea seems to follow socioeconomic and housing conditions.

### Stratification by age
The multiple logistic regressions showed that, at constant access to basic hygiene and sanitation, the living conditions (dwelling's materials and cooking space) were significantly associated with the risk of diarrhea in the general population. By order of significance:  
- ACCESS to BASIC HYGIENE was consistently and significantly associated with diarrhoea (lower risks) across age groups. This is certainly due to the fact that having basic hygiene amenities within premises facilitates handwashing, thus breaking pathways of exposure at key moments (e.g. before eating).
- SECONDARY EDUCATION was consistently and significantly associated with diarrhoea (increasing risk) across age groups. This is certainly due to a reporting bias within the higher socioeconomic stratum, who tends to report more accurately cases of diarrhoea - which has been observed elsewhere (DOI: 10.1093/ije/dym202). By extension, the same may apply to the level of ASSET-BASED WEALTH (another socioeconomic indicator). These two variables are considered here as confounders interfering in the recall bias.
- INADEQUATE HOUSING MATERIALS was consistently associated with diarrhoea (increasing risk) across age groups, but was only significant when analyzing the risk in the general population. Among children under five years old, it also likely that it is a risk factor (OR=2.06), but the lower CI value was just below 1 (lower CI=0.98). Living in a dwelling built with inadequate materials may exacerbate exposure to pathogens, especially in informal settlements where access to basic services is limited and environmental exposure relatively high.
- COOKING INDOORS was also consistently associated with diarrhoea (reducing risk) across age groups, but was only significant when analyzing the risk in the general population. Having an indoors cooking space certainly offers a better protection by, for example, separating spaces where the food is prepared from external contamination sources like flies or dirt.
- WATER AVAILABILITY in the month preceding the survey was also consistently associated with diarrhoea (reducing risk) across age groups, but was only significant when analyzing the risk in the general population. Certainly, constant access to water is a basic need and a requirement to implement basic hygiene.
- Surprisingly, access to basic sanitation had no association with diarrhoea. This may be related to limitations of data collected, which did not assess their maintenance (e.g., emptying of latrines and general management of excreta).

### Methodological note
- HAND-HYGIENE was not included in this step because it was only reported for the respondent (not for each indvidual, as was the case for diarrhea). In fact, hand-washing habits may vary significantly between individuals within a same household, and the respondent's "exposure" is certainly not representative of that of other household members. Therefore, this hand-washing is analized only in Notebook 1b - which consists of analyses at household-level.